### Examine a SWMM report file

In [ ]:
from swmm_read import *
#fname = "wingohocking_PWD_2012-13.rpt"
fname = "SWMM_modified.rpt"
rptFileStr = read_rpt_file(fname)
#print rptFileStr

### Read and display the report

In [ ]:
reportDict = read_report(rptFileStr)
reportDict

### Test SWMM runs from Jupyter 

In [ ]:
from swmm_run import *
swmmInpFileNameStr = "wingohocking_PWD_2011-12.inp"  
swmmRptFileNameStr = "wingohocking_PWD_2011-12.rpt"
timeStatsDict = runswmm(swmmInpFileNameStr,swmmRptFileNameStr)

### Select a watershed, a weather year, and find the best treatment ratio

In [ ]:
from swmm_read import *
from swmm_run import *
swmmRptFileNameStr = "wingohocking_PWD_2012-13.rpt"
swmmRptStr = read_rpt_file(swmmRptFileNameStr)
swmmRptDict = read_report(swmmRptStr)
outfallFlowTimeSeries = swmmRptDict["outflow_series"]
wingohockingImperviousArea = 3122.0  # acres
#ratioList = [0.075,0.080,0.085,0.090,0.095,0.10]
ratioList = [0.080,0.081,0.082,0.083,0.084,0.085]
for treatmentRatio in ratioList:   # CFS / impervious acre
    thresholdCFS = wingohockingImperviousArea*treatmentRatio
    outfallDict = calculate_cso(outfallFlowTimeSeries,thresholdCFS)
    csoVolumeMgal = outfallDict['cso_volume']/1e6
    totVolumeMgal = outfallDict['tot_volume']/1e6
    print "Treatment Ratio = %0.3f,  CSO Volume = %0.0f MGAL,  Total Volume = %0.0f MGAL" % (treatmentRatio,csoVolumeMgal,totVolumeMgal)

### SINGLE RUN: Scatter LID around subcatchments, calculate CSO, display results

In [ ]:
import yaml
from swmm_objects import *
from swmm_read import *
from swmm_run import *

squareFeetInOneAcre = 43560.0
swmmInpFileNameStr = "wingohocking_PWD_2012-13.inp"  # initial SWMM .inp file
treatmentRatio = 0.081   # cfs/imperv_acre calibrated for Wingohocking T-14 outfall 7/1/12 - 6/30/13
watershedImpervAreaAcre = total_impervious_area(swmmInpFileNameStr)
thresholdOutflowCFS = treatmentRatio*watershedImpervAreaAcre
lidDictFileNameStr = "LID_Dict.yaml"
lidShedsFileNameStr = "Wingohocking_LID_Sheds_List.yaml"
f = open(lidDictFileNameStr,'r')   # Read the no LID changable parameters from file
lidDict = yaml.load(f)  # list of dicts, one for each line in [LID_USAGE]
f.close()
f = open(lidShedsFileNameStr,'r')   # Read the no LID changable parameters from file
lidShedsList = yaml.load(f)  # list of dicts, one for each line in [LID_USAGE]
f.close()

swmmInpStr = read_inp_file(swmmInpFileNameStr)
(section_names,sections) = read_inp(swmmInpStr)
model = swmm_model("Model",section_names,sections)
swmmInputFileStr=model.output()

subcatAreaDict = {}
subcatImpPctDict = {}
subcatTreatableAreaDict = {}
for lidShed in lidShedsList:
    area = model.getSubcatchmentArea(lidShed)
    subcatAreaDict[lidShed] = area
    pctImperv = model.getSubcatchmentPctImperv(lidShed)
    subcatImpPctDict[lidShed] = pctImperv
    subcatTreatableAreaDict[lidShed] = area*pctImperv/100.0

lid = 'wakefield_BR_RG'

contribImpervAreaAcre = lidDict[lid]['ContribImpervArea'] / squareFeetInOneAcre  # acres
subcatMaxLID = {}
for lidShed in lidShedsList:
    maxLID = int(subcatTreatableAreaDict[lidShed]/contribImpervAreaAcre)
    subcatMaxLID[lidShed] = maxLID
    model.lidChangeNumber(lidShed,lid,maxLID)
    area = model.getLidArea(lidShed,lid)   # the lid footprint in the subcatchment (all subcats the same now)
    model.lidChangeArea(lidShed,lid,area,contribImpervAreaAcre)  # must call to properly set FromImp parameter !!
swmmInputFileStr=model.output()
swmmRptDict = swmm_run_return_results(swmmInputFileStr)
outfallFlowTimeSeries = swmmRptDict["outflow_seriesCFS"]
outfallDict = calculate_cso(outfallFlowTimeSeries,thresholdOutflowCFS)
csoVolumeMgal = outfallDict['cso_volume']/1e6
totVolumeMgal = outfallDict['tot_volume']/1e6
csoReductionMgal = 1552.0 - csoVolumeMgal
print "Treatment Ratio = %0.3f,  CSO Volume = %0.0f MGAL,  CSO Reduction = %0.0f MGAL" % (treatmentRatio,csoVolumeMgal,csoReductionMgal)

### SWMM input and report files associated with the single run are stored and can be displayed using "print"

In [ ]:
str = swmmRptDict['swmmInpFileStr']
#str = swmmRptDict['swmmRptFileStr']
print str